In [1]:
# Import all dependencies

import numpy as np
import os
from netCDF4 import Dataset
from numpy import sum 

In [2]:
def year1850_mmr(ghg_path, molar_mass, species):
    """
    Use the CMIP7 dataset for the given species
    to calculate the mass mixing ratio.
    """
    def scale(ghg_set, species):
        """
        Determine the scaling factor used for the given species
        """
        SCALE_LOOKUP = {
            "ppm": 1.0e-6,
            "ppb": 1.0e-9,
            "ppt": 1.0e-12}
        return SCALE_LOOKUP[ghg_set.variables[species].units]
    
    # Read in the CMIP7 dataset
    ghg_set = Dataset(ghg_path(species), mode="r")

    # Initialize the times array
    times = np.array(ghg_set["time"][:])

    # Find the index of the times array corresponding to the year 1850
    YEAR1850 = 182.5
    YEAR1850_INDEX = times.searchsorted(YEAR1850)
    assert times[YEAR1850_INDEX] == YEAR1850

    # Read in the concentration for the year 1850
    ghg_conc = np.array(ghg_set[species][:])
    year1850_ghg_conc = ghg_conc[YEAR1850_INDEX]

    # Determine the mass mixing ratio
    DRY_AIR_MOLAR_MASS = 28.97
    species_scale = scale(ghg_set, species)
    return year1850_ghg_conc * species_scale * molar_mass[species] / DRY_AIR_MOLAR_MASS

In [3]:
# Specify the CMIP7 dataset relative to species

INPUT4MIPS_DIR = "/g/data/qv56/replicas/input4MIPs"
GHG_FILE = lambda species: f"CMIP7/CMIP/CR/CR-CMIP-1-0-0/atmos/yr/{species}/gm/v20250228/{species}_input4MIPs_GHGConcentrations_CMIP_CR-CMIP-1-0-0_gm_1750-2022.nc"
GHG_PATH = lambda species: os.path.join(INPUT4MIPS_DIR, GHG_FILE(species))

In [4]:
# Specify the molar mass of each species in grams per mole

MOLAR_MASS_LOOKUP = {
    "co2": 44.01,
    "n2o": 44.01,
    "ch4": 16.04,
    "cfc11": 137.37,
    "cfc12": 120.91,
    "cfc113": 187.375,
    "hcfc22": 86.47,
    "hfc125": 120.02,
    "hfc134a": 102.03}

In [5]:
for species in MOLAR_MASS_LOOKUP.keys():
    species_mmr = year1850_mmr(GHG_PATH, MOLAR_MASS_LOOKUP, species)
    print(f"{species:8}: {species_mmr:12.6e}")

co2     : 4.318924e-04
n2o     : 4.125577e-07
ch4     : 4.422761e-07
cfc11   : 0.000000e+00
cfc12   : 0.000000e+00
cfc113  : 0.000000e+00
hcfc22  : 0.000000e+00
hfc125  : 0.000000e+00
hfc134a : 0.000000e+00
